In [ ]:
Хеш-функции и хеш-таблицы
Краснов Д.О.
Цель работы
Изучение хеш-функций и хеш-таблиц, а также основных операций над ними.

In [ ]:
Задание 1( Хеш-таблица на основе метода цепочек)

In [ ]:
class Node:
    def __init__(self, key, value):
        self.key = key
        self.value = value
        self.next = None


class HashTable:
    def __init__(self, capacity=10):
        self.capacity = capacity
        self.size = 0
        self.buckets = [None] * capacity
    
    def _hash(self, key):
        return hash(key) % self.capacity 
    
    def put(self, key, value):
        index = self._hash(key)  
        node = self.buckets[index]
        
        if node is None:
            self.buckets[index] = Node(key, value)
            self.size += 1
            return
        
        prev = None
        while node is not None:
            if node.key == key:
                node.value = value
                return
            prev = node
            node = node.next
        
        prev.next = Node(key, value)
        self.size += 1
    
    def get(self, key):
        """Возвращает значение по ключу или None если ключ не найден"""
        index = self._hash(key)
        node = self.buckets[index]
        
        while node is not None:
            if node.key == key:
                return node.value
            node = node.next
        
        return None
    
    def remove(self, key):
        index = self._hash(key)
        node = self.buckets[index]
        prev = None
        
        while node is not None:
            if node.key == key:
                if prev is None:
                    self.buckets[index] = node.next
                else:
                    prev.next = node.next
                self.size -= 1
                return True
            prev = node
            node = node.next
        
        return False
    
    def __len__(self):
        return self.size
    
    def __contains__(self, key):
        return self.get(key) is not None
    
    def __getitem__(self, key):
        value = self.get(key)
        if value is None:
            raise KeyError(f"Key '{key}' not found")
        return value
    
    def __setitem__(self, key, value):
        self.put(key, value)



def test_basic():
    ht = HashTable()
    ht.put("apple", 5)
    ht.put("banana", 10)
    
    assert ht.get("apple") == 5
    assert ht.get("banana") == 10
    assert ht.get("orange") is None
    assert len(ht) == 2
    print(" Базовая функциональность работает")


def test_update():
    ht = HashTable()
    ht.put("apple", 5)
    ht.put("apple", 15) 
    
    assert ht.get("apple") == 15
    assert len(ht) == 1
    print(" Обновление значений работает")


def test_remove():
    ht = HashTable()
    ht.put("apple", 5)
    ht.put("banana", 10)
    
    assert ht.remove("apple") == True
    assert ht.remove("apple") == False
    assert ht.get("apple") is None
    assert len(ht) == 1
    print(" Удаление элементов работает")


def test_operators():
    ht = HashTable()
    ht["apple"] = 5
    ht["banana"] = 10
    
    assert ht["apple"] == 5
    assert "apple" in ht
    assert "orange" not in ht
    print(" Python операторы работают")


def test_collisions():
    ht = HashTable(2)  
    ht.put("a", 1)
    ht.put("b", 2)
    ht.put("c", 3)
    
    assert ht.get("a") == 1
    assert ht.get("b") == 2
    assert ht.get("c") == 3
    assert len(ht) == 3
    print(" Обработка коллизий работает")


if __name__ == "__main__":
    test_basic()
    test_update()
    test_remove()
    test_operators()
    test_collisions()
    print("\n Все тесты пройдены!")

In [ ]:
Задание 2 (Хеш-таблица на основе открытой адресации)

In [ ]:
class HashTableOpenAddressing:    
    def __init__(self, size=10):
        self.size = size
        self.table = [None] * size
        self.count = 0
        self.load_factor_threshold = 0.7 # Порог для перехэширования
        self.DELETED = object() #DELETED = object() - специальный маркер, который отличает удаленные элементы от пустых ячеек
    
    def _hash(self, key):
        if isinstance(key, int):
            return key % self.size
        elif isinstance(key, str): #полиномиальный хеш (31 - простое число для уменьшения коллизий)
            hash_value = 0
            for char in key:
                hash_value = (hash_value * 31 + ord(char)) % self.size # hash = (97*31 + 98) % size = (3007 + 98) % size
            return hash_value 
        else:
            return hash(key) % self.size
    
    def _probe(self, key, i):
        return (self._hash(key) + i) % self.size
    
    def _resize(self):
        if self.count / self.size > self.load_factor_threshold:
            old_table = self.table
            old_size = self.size
            self.size = self.size * 2
            self.table = [None] * self.size
            self.count = 0
            for item in old_table:
                if item and item != self.DELETED:
                    self.put(item[0], item[1])
    
    def put(self, key, value):
        self._resize()
        for i in range(self.size):
            index = self._probe(key, i)
            if self.table[index] is None or self.table[index] == self.DELETED:
                self.table[index] = (key, value)
                self.count += 1
                return
            elif self.table[index][0] == key:
                self.table[index] = (key, value)
                return
        raise Exception("Hash table is full")
    
    def get(self, key):
        for i in range(self.size):
            index = self._probe(key, i)
            if self.table[index] is None:
                break
            elif self.table[index] != self.DELETED and self.table[index][0] == key:
                return self.table[index][1]
        raise KeyError(f"Key {key} not found")
    
    def delete(self, key):
        for i in range(self.size):
            index = self._probe(key, i)
            if self.table[index] is None:
                break
            elif self.table[index] != self.DELETED and self.table[index][0] == key:
                self.table[index] = self.DELETED #Если бы ставили None, то цепочка пробирования прервалась бы и последующие элементы стали бы недоступны
                self.count -= 1
                return True
        raise KeyError(f"Key {key} not found")
    
    def contains(self, key):
        try:
            self.get(key)
            return True
        except KeyError:
            return False
    
    def __str__(self):
        result = []
        for i, item in enumerate(self.table):
            if item is None:
                result.append(f"Bucket {i}: None")
            elif item == self.DELETED:
                result.append(f"Bucket {i}: DELETED")
            else:
                result.append(f"Bucket {i}: ({item[0]}: {item[1]})")
        return "\n".join(result)



def test_basic_operations():
    ht = HashTableOpenAddressing()
    ht.put("apple", 5)
    ht.put("banana", 10)
    
    assert ht.get("apple") == 5
    assert ht.get("banana") == 10
    assert ht.count == 2
    print("Базовая функциональность работает")


def test_update_values():
    ht = HashTableOpenAddressing()
    ht.put("apple", 5)
    ht.put("apple", 15)  
    
    assert ht.get("apple") == 15
    assert ht.count == 1  
    print(" Обновление значений работает")


def test_deletion():
    ht = HashTableOpenAddressing()
    ht.put("apple", 5)
    ht.put("banana", 10)
    
    assert ht.delete("apple") == True
    assert ht.count == 1
    try:
        ht.get("apple")
        assert False, "Должно было возникнуть KeyError"
    except KeyError:
        pass
    
    assert ht.contains("banana") == True
    assert ht.contains("apple") == False
    print(" Удаление элементов работает")


def test_collisions():
    ht = HashTableOpenAddressing(5) 
    ht.put("a", 1)
    ht.put("b", 2)
    ht.put("c", 3)
    ht.put("d", 4)
    
    assert ht.get("a") == 1
    assert ht.get("b") == 2
    assert ht.get("c") == 3
    assert ht.get("d") == 4
    assert ht.count == 4
    
    
    ht.delete("b")
    assert ht.contains("b") == False
    ht.put("e", 5)
    assert ht.get("e") == 5
    print(" Обработка коллизий работает")



if __name__ == "__main__":
    test_basic_operations()
    test_update_values()
    test_deletion()
    test_collisions()
    
    print("\n Все тесты хеш-таблицы с открытой адресацией пройдены!")

In [ ]:
Задание 3(Блокчейн)

In [ ]:
import hashlib
import time

class Block:
    def __init__(self, index, timestamp, data, previous_hash):
        self.index = index           # Порядковый номер блока
        self.timestamp = timestamp   # Время создания блока
        self.data = data             # Полезная нагрузка (транзакции, информация)
        self.previous_hash = previous_hash  # Хеш предыдущего блока
        self.hash = self.calculate_hash()   # Хеш текущего блока
        
    def calculate_hash(self): # Вычисление хеша
        block_string = f"{self.index}{self.timestamp}{self.data}{self.previous_hash}" #Создание строки блока
        return hashlib.sha256(block_string.encode()).hexdigest()
    
    def __str__(self):
        return f"Block {self.index} [Hash: {self.hash}, Previous: {self.previous_hash}, Data: {self.data}]"

class Blockchain:    
    def __init__(self):
        self.chain = [self.create_genesis_block()]
    
    def create_genesis_block(self):
        return Block(0, time.time(), "Genesis Block", "0")
    
    def add_block(self, data):
        previous_block = self.chain[-1]
        new_block = Block(
            index=len(self.chain),
            timestamp=time.time(),
            data=data,
            previous_hash=previous_block.hash
        )
        self.chain.append(new_block)
    
    def is_chain_valid(self):
        for i in range(1, len(self.chain)):
            current_block = self.chain[i]
            previous_block = self.chain[i-1]
            if current_block.hash != current_block.calculate_hash():
                return False
            if current_block.previous_hash != previous_block.hash:
                return False
        return True
    
    def get_latest_block(self):
        return self.chain[-1]
    
    def get_chain_length(self):
        return len(self.chain)
    
    def __str__(self):
        return "\n".join(str(block) for block in self.chain)



def test_genesis_block():
    blockchain = Blockchain()
    
    assert len(blockchain.chain) == 1
    assert blockchain.chain[0].index == 0
    assert blockchain.chain[0].data == "Genesis Block"
    assert blockchain.chain[0].previous_hash == "0"
    assert blockchain.chain[0].hash == blockchain.chain[0].calculate_hash()
    print("✓ Генезис-блок создан корректно")


def test_adding_blocks():
    blockchain = Blockchain()
    
    blockchain.add_block("Transaction 1")
    blockchain.add_block("Transaction 2")
    
    assert len(blockchain.chain) == 3
    assert blockchain.chain[1].data == "Transaction 1"
    assert blockchain.chain[2].data == "Transaction 2"
    assert blockchain.chain[1].index == 1
    assert blockchain.chain[2].index == 2
    print(" Добавление блоков работает")


def test_block_linking():
    blockchain = Blockchain()
    
    blockchain.add_block("Data 1")
    blockchain.add_block("Data 2")
    
    
    assert blockchain.chain[1].previous_hash == blockchain.chain[0].hash
    assert blockchain.chain[2].previous_hash == blockchain.chain[1].hash
    print(" Связи между блоками корректны")


def test_chain_validation():
    blockchain = Blockchain()
    
    blockchain.add_block("Valid Data 1")
    blockchain.add_block("Valid Data 2")
    
    
    assert blockchain.is_chain_valid() == True
    
   
    blockchain.chain[1].data = "Tampered Data"
    
    
    assert blockchain.is_chain_valid() == False
    print(" Валидация цепочки работает")


def test_data_integrity():
    blockchain = Blockchain()
    
    original_data = "Original Data"
    blockchain.add_block(original_data)
    
    original_hash = blockchain.chain[1].hash
    
   
    assert blockchain.chain[1].calculate_hash() == original_hash
    
   
    blockchain.chain[1].data = "Modified Data"
    assert blockchain.chain[1].calculate_hash() != original_hash
    print(" Целостность данных и хеширование работают")


if __name__ == "__main__":
    test_genesis_block()
    test_adding_blocks()
    test_block_linking()
    test_chain_validation()
    test_data_integrity()
    print("\n Все тесты блокчейна пройдены!")

In [ ]:
Задание 4(Проверка пересечения двух массивов)

In [ ]:
def has_intersection(arr1, arr2):
    
    hash_set = set(arr1) #Создаем множество(O(n) - для хранения множества)
    for item in arr2:
        if item in hash_set:
            return True
    return False



def test_has_intersection():
   
    arr1 = [1, 2, 3, 4, 5]
    arr2 = [5, 6, 7, 8, 9]
    assert has_intersection(arr1, arr2) == True
    print(" Тест 1: Есть пересечение - пройден")
    
   
    arr1 = [1, 2, 3, 4]
    arr2 = [5, 6, 7, 8]
    assert has_intersection(arr1, arr2) == False
    print(" Тест 2: Нет пересечения - пройден")
    
    arr1 = []
    arr2 = [1, 2, 3]
    assert has_intersection(arr1, arr2) == False
    
    arr1 = [1, 2, 3]
    arr2 = []
    assert has_intersection(arr1, arr2) == False
    
    arr1 = []
    arr2 = []
    assert has_intersection(arr1, arr2) == False
    print(" Тест 3: Пустые массивы - пройден")
    
    arr1 = [1, 2, 3, 4, 5]
    arr2 = [3, 4, 5, 6, 7]
    assert has_intersection(arr1, arr2) == True
    print(" Тест 4: Множественные пересечения - пройден")
    
    arr1 = ["apple", "banana", "orange"]
    arr2 = ["banana", "grape", "kiwi"]
    assert has_intersection(arr1, arr2) == True
    
    arr1 = [1.5, 2.7, 3.1]
    arr2 = [3.1, 4.2, 5.8]
    assert has_intersection(arr1, arr2) == True
    print(" Тест 5: Разные типы данных - пройден")

if __name__ == "__main__":
    test_has_intersection()
   
    print("\n Все тесты пройдены успешно!")

In [ ]:
Задание 5 (Проверка уникальности элементов в массиве)

In [ ]:
def has_unique_elements(arr):
    
    hash_set = set() # Пустое множество
    for item in arr:
        if item in hash_set:
            return False
        hash_set.add(item)
    return True


def test_has_unique_elements():
    
   
    arr = [1, 2, 3, 4, 5]
    assert has_unique_elements(arr) == True
    print(" Тест 1: Все элементы уникальны - пройден")
    
    
    arr = [1, 2, 3, 2, 4]
    assert has_unique_elements(arr) == False
    print(" Тест 2: Есть дубликаты - пройден")
    
    
    arr = []
    assert has_unique_elements(arr) == True
    print(" Тест 3: Пустой массив - пройден")
    
    
    arr = [42]
    assert has_unique_elements(arr) == True
    print(" Тест 4: Один элемент - пройден")
    
    
    arr = ["apple", "banana", "orange"]
    assert has_unique_elements(arr) == True
    
    arr = ["apple", "banana", "apple"]
    assert has_unique_elements(arr) == False
    
    arr = [1, "1", 2, "2"]  #
    assert has_unique_elements(arr) == True
    print(" Тест 5: Разные типы данных - пройден")


if __name__ == "__main__":
    test_has_unique_elements()
    print("\n Все тесты пройдены успешно!")

In [ ]:
Задание 6 (Нахождение пар с заданной суммой)

In [ ]:
def find_pairs_with_sum(arr, target_sum):
    result = []
    seen = set() # Множество для отслеживания просмотренных чисел
    for num in arr:
        complement = target_sum - num
        if complement in seen:
            result.append((complement, num))
        seen.add(num)
    return result

def test_find_pairs_with_sum():
    """Тестирование функции find_pairs_with_sum"""
    arr = [2, 7, 11, 15, 3, 6]
    target = 9
    result = find_pairs_with_sum(arr, target) 
    
    
    assert (2, 7) in result
    assert (3, 6) in result
    assert len(result) == 2
    print(" Тест пройден: найдены пары (2,7) и (3,6)")

if __name__ == "__main__":
    test_find_pairs_with_sum()
    print(" Тест пройден успешно!")

In [ ]:
Задание 7(Задача на проверку анаграмм)

In [ ]:
def are_anagrams(str1, str2):
    if len(str1) != len(str2):
        return False
    char_count = {} #Создание словаря для подсчета символов
    for char in str1:
        if char in char_count:
            char_count[char] += 1
        else:
            char_count[char] = 1
    for char in str2:
        if char not in char_count or char_count[char] == 0:
            return False
        char_count[char] -= 1
    return True


def test_are_anagrams():
    """Тестирование функции are_anagrams"""
    str1 = "listen"
    str2 = "silent"
    result = are_anagrams(str1, str2)
    
    assert result == True
    print(" Тест пройден: 'listen' и 'silent' являются анаграммами")

if __name__ == "__main__":
    test_are_anagrams()
    print(" Тест пройден успешно!")